In [3]:
"""
This code is used to assign retrofit
functions to buildings based on the vulnerability
model currently assigned to them.

retrofit_percent can be changed to whatever rate of
retrofit uptake is required.
"""

import os
import pandas as pd

# read in exposure data
datapath = r"X:\georisk\HaRIA_B_Wind\projects\qfes_swha\data" \
           r"\derived\exposure\2021"
filename = "SEQ_ResidentialExposure_NEXIS_2021_M4_updated_v2.csv"
df = pd.read_csv(os.path.join(datapath, filename))

# read in retrofit eligability data
datapath2 = r"X:\georisk\HaRIA_B_Wind\projects\qfes_swha\data\8. USERS\u12161\retrofit"
filename2 = "targeted_retrofit_eligibility.csv"
df2 = pd.read_csv(os.path.join(datapath2, filename2),)
df2.set_index(["SA2_MAIN16"], inplace=True)

In [7]:
# merge df and df2
# Rather than an inner join, we use an outer join to merge the dataframes - this covers the situation where there are duplicate 
# SA2_MAIN16 values in the eligibility data. 
df_retro = pd.merge(df, df2[[ "Targeted_Retrofit"]], left_on=["SA2_CODE"], right_index=True, how="outer", sort=False)
df_retro = df_retro.sort_values("Targeted_Retrofit").drop_duplicates()
df_retro = df_retro[~df_retro['Targeted_Retrofit'].isna()]

In [8]:

retrofit_percent_new = 0.113
retrofit_percent_old = 0.819

# legacy buildings original vuln functions: retrofit functions (windows)
# dic_old_RF1 = {
#    'dw350': 'dw450',
#    'dw351': 'dw451',
#    'dw352': 'dw452',
# }

# legacy buildings original vuln functions: retrofit functions (structural)
# dic_old_RF2 = {
#    'dw350': 'dw550',
#    'dw351': 'dw551',
#    'dw352': 'dw552',
#  }

# legacy buildings original vuln functions: retrofit functions (wind and struc)
dic_old_RF3 = {
    'dw350': 'dw650',
    'dw351': 'dw651',
    'dw352': 'dw652',
}

# modern buildings original vuln functions: retrofit functions (windows)
dic_new = {
    'dw353': 'dw453',
    'dw354': 'dw454',
    'dw355': 'dw455',
    'dw356': 'dw456',
    'dw357': 'dw457',
    'dw358': 'dw458',
    'dw359': 'dw459',
    'dw360': 'dw460',
    'dw361': 'dw461',
    'dw362': 'dw462',
    'dw363': 'dw463',
    'dw364': 'dw464',
}

retrofit_condition = ["Eligible"]
retrofit_condition2 = ["N3", "N4"]

# create a list of the current vuln function so that the can be used
# to find buildings assigned these functions
# list_old_RF1 = list(dic_old_RF1.keys())
# list_old_RF2 = list(dic_old_RF2.keys())
list_old_RF3 = list(dic_old_RF3.keys())
list_new = list(dic_new.keys())
list_retrofit = list(retrofit_condition)
list_retrofit2 = list(retrofit_condition2)

df_retro = df_retro.rename({'WIND_VULNERABILITY_FUNCTION_ID': 'Wind_func'}, axis='columns')



In [9]:
# find relevent buildings and sample a percent of them (retrofit_percent)
df_new = df_retro[(df_retro['Targeted_Retrofit'].isin(list_retrofit))]
df_new = df_new[(df_new['AS4055_CLASS'].isin(list_retrofit2))]
df_new = df_new[(df_new['Wind_func'].isin(list_new))]
df_new = df_new.sample(frac=retrofit_percent_new)

# three retrofit options with equal probability
# df_old_RF1 = df_retro[(df_retro['Targeted_Retrofit'].isin(list_retrofit))]
# df_old_RF1 = df_old_RF1[(df_old_RF1['Wind_func'].isin(list_old_RF1))]
# df_old_RF1 = df_old_RF1.sample(frac=(retrofit_percent_old/3))

# df_old_RF2 = df_retro[(df_retro['Targeted_Retrofit'].isin(list_retrofit))]
# df_old_RF2 = df_old_RF2[(df_old_RF2['Wind_func'].isin(list_old_RF2))]
# df_old_RF2 = df_old_RF2.sample(frac=(retrofit_percent_old/3))

df_old_RF3 = df_retro[(df_retro['Targeted_Retrofit'].isin(list_retrofit))]
df_old_RF3 = df_old_RF3[(df_old_RF3['AS4055_CLASS'].isin(list_retrofit2))]
df_old_RF3 = df_old_RF3[(df_old_RF3['Wind_func'].isin(list_old_RF3))]
df_old_RF3 = df_old_RF3.sample(frac=(retrofit_percent_old))



In [10]:
df_retro

,Unnamed: 0,AS4055_CLASS,ROOF_TYPE,YEAR_BUILT,LID,LATITUDE,LONGITUDE,MB_CODE,MB_CAT,SA1_CODE,...,WALL_TYPE,REPLACEMENT_VALUE,CONTENTS_VALUE,WIND_VULNERABILITY_MODEL_NUMBER,Wind_func,WIND_REGION_CLASSIFCATION,WIND_VULNERABILITY_SET,M4,M42,Targeted_Retrofit
615692,615692,N2,Metal Sheeting,1947 - 1961,GNAF_GAQLD156035173,-27.355451,153.042354,30563178000,Residential,30204104608,...,Timber,551097,75600,268,dw350,B,wind_vulnerability_2012,0.630110,0,Eligible
447235,447235,N2,Metal Sheeting,1840 - 1890,GNAF_GAQLD154958504,-27.507323,153.032848,30029140000,Residential,30302105214,...,Timber,610143,75600,298,dw350,B,wind_vulnerability_2012,0.672476,0,Eligible
447231,447231,N2,Tiles,1840 - 1890,GNAF_GAQLD154958507,-27.507294,153.032648,30029140000,Residential,30302105214,...,Brick Veneer,642715,75600,271,dw351,B,wind_vulnerability_2012,0.672476,0,Eligible
447225,447225,N2,Metal Sheeting,1840 - 1890,GNAF_GAQLD156196474,-27.507232,153.031625,30029160000,Residential,30302105217,...,Timber,456062,75600,298,dw350,B,wind_vulnerability_2012,0.651848,0,Eligible
447222,447222,N2,Metal Sheeting,1840 - 1890,GNAF_GAQLD156198221,-27.507212,153.029099,30030050000,Residential,30302105217,...,Timber,354015,75600,298,dw350,B,wind_vulnerability_2012,0.618460,0,Eligible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506242,506242,N2,Metal Sheeting,1997 - present,GNAF_GAQLD155578227,-27.531749,153.069993,30097740000,Residential,30303106320,...,Fibro / asbestos cement sheeting,278717,75600,131,dw354,B,wind_vulnerability_2012,0.613518,0,Not Eligible
506243,506243,N2,Metal Sheeting,1997 - present,GNAF_GAQLD425769217,-27.531834,153.070057,30097740000,Residential,30303106320,...,Fibro / asbestos cement sheeting,279554,75600,131,dw354,B,wind_vulnerability_2012,0.612636,0,Not Eligible
506244,506244,N2,Tiles,1997 - present,GNAF_GAQLD424939975,-27.531942,153.078874,30563933200,Residential,30303106329,...,Double Brick,267478,39300,135,dw360,B,wind_vulnerability_2012,0.634376,0,Not Eligible
506222,506222,N2,Metal Sheeting,1997 - present,GNAF_GAQLD425119904,-27.528374,153.086113,30098500000,Residential,30303106314,...,Timber,350182,75600,130,dw354,B,wind_vulnerability_2012,0.651794,0,Not Eligible


In [11]:
# change original vuln func to retrofit vuln func for the selected
# retrofit_percent
df_new['Wind_func'] = df_new['Wind_func'].map(dic_new)
# df_old_RF1['Wind_func'] = df_old_RF1['Wind_func'].map(dic_old_RF1)
# df_old_RF2['Wind_func'] = df_old_RF2['Wind_func'].map(dic_old_RF2)
df_old_RF3['Wind_func'] = df_old_RF3['Wind_func'].map(dic_old_RF3)

# overwrite the original datafrom with the retrofitted buildings
df_retro.update(df_new)
# df_retro.update(df_old_RF1)
# df_retro.update(df_old_RF2)
df_retro.update(df_old_RF3)

# save
outputfile = "Targeted_Retrofit_v2.csv"
df_retro.to_csv(os.path.join(datapath2, outputfile))
